# Forward propagation

In [7]:
import numpy as np
import tensorflow as tf

In [8]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

X_train, Y_train = load_coffee_data()
norm = tf.keras.layers.Normalization(axis=-1)
norm.adapt(X_train)
X_train = norm(X_train)
m = X_train.shape[0] # number of training examples
n = X_train.shape[1] # number of features

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(x))

def getActivationDerivative(activation):
    if activation == 'linear':
        return lambda x : x

    if activation == 'sigmoid':
        return lambda x : sigmoid(x) * (1 - sigmoid(x))
    
    if activation == 'relu':
        return lambda x : x if x > 0 else 0

In [10]:
class Dense:
    def __init__(self, units, activation=None):
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        self.activationDerivative = getActivationDerivative(activation)
        self.W = None
        self.b = None
        self.built = False
    
    def build(self, input_shape):
        self.W = np.random.uniform(-1, 1,(input_shape, self.units))
        self.b = np.random.randint(-5,5,(self.units,))
        self.built = True
    
    def get_weights(self):
        return (self.W, self.b)
    
    def call(self, a_in):
        if self.built == False:
            raise Error("Layer has not been built yet")
        
        Z = np.dot(np.transpose(self.W), a_in) + self.b
        return self.activation(Z)

In [11]:
class Sequential:
    def __init__(self, layers):
        self.input_shape = None;
        self.layers = layers
        self.output_layer = layers[-1]
        self.built = False;
        self.errorHist = [];
        
    def build(self, X):
        a_in = X[0]
        size = self.input_shape or a_in.shape[0]
        for layer in self.layers:
            layer.build(size)
            a_in = layer.call(a_in)
            size = a_in.shape[0]
        self.built = True
     
    def forwardProp(self, x):
        a_in = x
        for layer in self.layers:
            a_in = layer.call(a_in)
        a_out = a_in
        return a_out
    
    def backwardProp(self, y_hat, y):
        batch_size = len(y_hat)
        delta_L = np.zeros(self.output_layer.units)
        cost_diff = np.zeros(y_hat.shape[0])
        for i in range(batch_size):
            cost_diff += y_hat[i] - y[i]
        cost_diff /= batch_size
        delta_L = np.multiply(cost_diff, self.output_layer.activationDerivative())
        return cost
        
    def predict(self, x):
        if not (self.built):
            raise Error('Model has not been built yet')
        return self.forwardProp(x)
    
    def __str__(self):
        print('\nSequential\n')
        for layer in self.layers:
            if layer.built:
                print('W shape:', layer.get_weights()[0].shape)
                print('b shape:', layer.get_weights()[1].shape)
                print('\n')
        
        return ''

In [12]:
model = Sequential([
    Dense(10, activation='relu'),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.build(X_train)
def test(model, X, y, size):
    cost = 0
    for i in range(size):
        print('t: {},  d: {}'.format(X[i][0], X[i][1]))
        prediction = model.predict(X[i])
        expected = y[i]
        print('{} vs. {}'.format(0 if prediction < 0.5 else 1, int(expected[0])))
        print()
        cost += (expected - prediction)**2
    return cost
# test(model, X_train, Y_train, 10)
predictions = []
for i in range(10):
    predictions.append(model.predict(X_train[i]))

# print(model.backwardProp(predictions, Y_train[:10]))